In [1]:
import requests
from bs4 import BeautifulSoup

In [2]:
# define the url that we are going to start scraping
standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"

In [3]:
# will make a request to the server, and download the html of this page
standings_home_page = requests.get(standings_url)

In [4]:
#from bs4 import BeautifulSoup

In [5]:
# initialize soup object
soup = BeautifulSoup(standings_home_page.text)
# grab the first element/instance of a stats_table in the html document, helps narrow down what we are looking for
standings_table = soup.select('table.stats_table')[0]
#standings_table = soup.select('table.stats_table')
#standings_table = soup.find('table', class_='stats_table')
# find all of the a tags, find_all only finds tags (the a tags contain href property, which each link has)
links = standings_table.find_all('a')
# after finding all the a elements inside the table, loop through every element and grab the href property
links = [l.get("href") for l in links]
# only include links that have 'squads' in the link
links = [l for l in links if '/squads/' in l]

In [6]:
standings_home_page.text

'    \n      \n<!DOCTYPE html>\n<html data-version="klecko-" data-root="/home/fb/deploy/www/base" lang="en" class="no-js" >\n<head>\n    <meta charset="utf-8">\n    <meta http-equiv="x-ua-compatible" content="ie=edge">\n    <meta name="viewport" content="width=device-width, initial-scale=1.0, maximum-scale=2.0" />\n    <link rel="dns-prefetch" href="https://cdn.ssref.net/req/202408232" />\n<script>\n/* https://docs.osano.com/hc/en-us/articles/22469433444372-Google-Consent-Mode-v2  */\n  window.dataLayer = window.dataLayer ||[];\n      function gtag(){dataLayer.push(arguments);}\n      gtag(\'consent\',\'default\',{\n        \'ad_storage\':\'denied\',\n        \'analytics_storage\':\'denied\',\n        \'ad_user_data\':\'denied\',\n        \'ad_personalization\':\'denied\',\n        \'personalization_storage\':\'denied\',\n        \'functionality_storage\':\'granted\',\n        \'security_storage\':\'granted\',\n        \'wait_for_update\': 500\n      });\n      gtag("set", "ads_data_re

In [7]:
# are only partial urls
links

['/en/squads/b8fd03ef/Manchester-City-Stats',
 '/en/squads/d07537b9/Brighton-and-Hove-Albion-Stats',
 '/en/squads/18bb7c10/Arsenal-Stats',
 '/en/squads/822bd0ba/Liverpool-Stats',
 '/en/squads/361ca564/Tottenham-Hotspur-Stats',
 '/en/squads/b2b47a98/Newcastle-United-Stats',
 '/en/squads/e4a775cb/Nottingham-Forest-Stats',
 '/en/squads/cff3d9bb/Chelsea-Stats',
 '/en/squads/7c21e445/West-Ham-United-Stats',
 '/en/squads/19538871/Manchester-United-Stats',
 '/en/squads/fd962109/Fulham-Stats',
 '/en/squads/8602292d/Aston-Villa-Stats',
 '/en/squads/cd051869/Brentford-Stats',
 '/en/squads/4ba7cbea/Bournemouth-Stats',
 '/en/squads/a2d435b3/Leicester-City-Stats',
 '/en/squads/33c895d4/Southampton-Stats',
 '/en/squads/47c64c55/Crystal-Palace-Stats',
 '/en/squads/b74092de/Ipswich-Town-Stats',
 '/en/squads/8cec06e1/Wolverhampton-Wanderers-Stats',
 '/en/squads/d3fd31cc/Everton-Stats']

In [8]:
# using a format string to turn our links into full urls (are absolute links)
team_urls = [f"https://fbref.com{l}" for l in links]

In [9]:
team_urls

['https://fbref.com/en/squads/b8fd03ef/Manchester-City-Stats',
 'https://fbref.com/en/squads/d07537b9/Brighton-and-Hove-Albion-Stats',
 'https://fbref.com/en/squads/18bb7c10/Arsenal-Stats',
 'https://fbref.com/en/squads/822bd0ba/Liverpool-Stats',
 'https://fbref.com/en/squads/361ca564/Tottenham-Hotspur-Stats',
 'https://fbref.com/en/squads/b2b47a98/Newcastle-United-Stats',
 'https://fbref.com/en/squads/e4a775cb/Nottingham-Forest-Stats',
 'https://fbref.com/en/squads/cff3d9bb/Chelsea-Stats',
 'https://fbref.com/en/squads/7c21e445/West-Ham-United-Stats',
 'https://fbref.com/en/squads/19538871/Manchester-United-Stats',
 'https://fbref.com/en/squads/fd962109/Fulham-Stats',
 'https://fbref.com/en/squads/8602292d/Aston-Villa-Stats',
 'https://fbref.com/en/squads/cd051869/Brentford-Stats',
 'https://fbref.com/en/squads/4ba7cbea/Bournemouth-Stats',
 'https://fbref.com/en/squads/a2d435b3/Leicester-City-Stats',
 'https://fbref.com/en/squads/33c895d4/Southampton-Stats',
 'https://fbref.com/en/squ

In [10]:
# Old: navigating the teams (Manchester city's - index 0) info website
# New: navigating the teams (Arsenal/Newcastle) info website (indexes can vary based off of team table placement/ranking)
newcastle_data = requests.get(team_urls[7])

In [11]:
# Goal: retrieve/store the Manchester City/Arsenal/Newcastle "Score & Fixtures" table in a pandas dataframe
import pandas as pd
from io import StringIO

# Assuming data.text contains your HTML content as a string
html_content = StringIO(newcastle_data.text)
# match looks for a specific string inside the table, pd.read_html reads all the tables on the page, turns the selected table into a pandas dataframe
matches = pd.read_html(html_content, match="Scores & Fixtures")[0]

In [12]:
matches.head()

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,xG,xGA,Poss,Attendance,Captain,Formation,Opp Formation,Referee,Match Report,Notes
0,2024-08-18,16:30,Premier League,Matchweek 1,Sun,Home,L,0.0,2.0,Manchester City,1.0,0.8,48.0,39818.0,Enzo Fernández,4-2-3-1,3-2-4-1,Anthony Taylor,Match Report,NaN
1,2024-08-22,20:00,Conf Lg,Play-off round,Thu,Home,W,2.0,0.0,ch Servette FC,NaN,NaN,59.0,NaN,Moisés Caicedo,4-3-3,4-2-3-1,Jérémie Pignard,Match Report,Leg 1 of 2
2,2024-08-25,14:00,Premier League,Matchweek 2,Sun,Away,W,6.0,2.0,Wolves,1.6,1.9,60.0,31235.0,Enzo Fernández,4-2-3-1,4-2-3-1,Darren England,Match Report,NaN
3,2024-08-29,20:30,Conf Lg,Play-off round,Thu,Away,NaN,NaN,NaN,ch Servette FC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Head-to-Head,Leg 2 of 2; Chelsea 2:0 Servette FC
4,2024-09-01,13:30,Premier League,Matchweek 3,Sun,Home,NaN,NaN,NaN,Crystal Palace,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Head-to-Head,NaN


In [13]:
# Goal: trying to obtain the link for "Shooting" under "Match Log Types"

# initialize a Beautiful soup instance, passing our html
soup = BeautifulSoup(newcastle_data.text)
# find all of the links on the page
links = soup.find_all('a')
# using list comprehension to get all the urls for the links
links = [l.get("href") for l in links]
# use another list comprehension to only find the link to the shooting stats (there are duplicate links)
links = [l for l in links if l and 'all_comps/shooting/' in l]

In [14]:
# the links displayed are relative urls, need to add the beginning portion "https://freb/" to download the data 
links

['/en/squads/cff3d9bb/2024-2025/matchlogs/all_comps/shooting/Chelsea-Match-Logs-All-Competitions',
 '/en/squads/cff3d9bb/2024-2025/matchlogs/all_comps/shooting/Chelsea-Match-Logs-All-Competitions',
 '/en/squads/cff3d9bb/2024-2025/matchlogs/all_comps/shooting/Chelsea-Match-Logs-All-Competitions',
 '/en/squads/cff3d9bb/2024-2025/matchlogs/all_comps/shooting/Chelsea-Match-Logs-All-Competitions']

In [15]:
# grab the html for the specific link, will download the html for the shooting stats page
newcastle_shooting_data = requests.get(f"https://fbref.com{links[0]}")

In [16]:
f"https://fbref.com{links[0]}"

'https://fbref.com/en/squads/cff3d9bb/2024-2025/matchlogs/all_comps/shooting/Chelsea-Match-Logs-All-Competitions'

In [17]:
# Assuming data.text contains your HTML content as a string
html_content = StringIO(newcastle_shooting_data.text)
# retrieve the "Shooting" table
shooting = pd.read_html(html_content, match="Shooting")[0]

In [18]:
'''
Description: As you can see there is a multi-level index, with "For Manchester City", "Standard", and "Expected" as 
the columns for the top level index. There exists 2 indices in the shooting table "For Manchester City, ..., Expected" 
and "Data Comp,..., np.G-xG, match report". 

Goal: We are removing this top level so we can access the dataframe by column.
'''
shooting.head()

For Chelsea                                                                \
         Date   Time            Comp           Round  Day Venue Result   GF   
0  2024-08-18  16:30  Premier League     Matchweek 1  Sun  Home      L  0.0   
1  2024-08-22  20:00         Conf Lg  Play-off round  Thu  Home      W  2.0   
2  2024-08-25  14:00  Premier League     Matchweek 2  Sun  Away      W  6.0   
3         NaN    NaN             NaN             NaN  NaN   NaN     --  NaN   

                         ... Standard               Expected               \
    GA         Opponent  ...     Dist   FK PK PKatt       xG npxG npxG/Sh   
0  2.0  Manchester City  ...     17.1  0.0  0     0      1.0  1.0    0.10   
1  0.0   ch Servette FC  ...      NaN  NaN  1     1      NaN  NaN     NaN   
2  2.0           Wolves  ...     15.4  1.0  0     0      1.6  1.6    0.12   
3  NaN              NaN  ...     16.1  1.0  1     1      NaN  NaN    0.11   

               Unnamed: 25_level_0  
  G-xG np:G-xG        Match Report  
0 -1.0    -1.0        Match Report  
1  NaN     NaN        Match Report  
2  4.4     4.4        Match Report  
3  8.0     7.0                 NaN  

[4 rows x 26 columns]

In [19]:
# drops the top level index
shooting.columns = shooting.columns.droplevel()

In [20]:
# top level index is gone
shooting.head()

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,...,Dist,FK,PK,PKatt,xG,npxG,npxG/Sh,G-xG,np:G-xG,Match Report
0,2024-08-18,16:30,Premier League,Matchweek 1,Sun,Home,L,0.0,2.0,Manchester City,...,17.1,0.0,0,0,1.0,1.0,0.10,-1.0,-1.0,Match Report
1,2024-08-22,20:00,Conf Lg,Play-off round,Thu,Home,W,2.0,0.0,ch Servette FC,...,NaN,NaN,1,1,NaN,NaN,NaN,NaN,NaN,Match Report
2,2024-08-25,14:00,Premier League,Matchweek 2,Sun,Away,W,6.0,2.0,Wolves,...,15.4,1.0,0,0,1.6,1.6,0.12,4.4,4.4,Match Report
3,NaN,NaN,NaN,NaN,NaN,NaN,--,NaN,NaN,NaN,...,16.1,1.0,1,1,NaN,NaN,0.11,8.0,7.0,NaN


In [21]:
# Combine the "matches" and "shooting" dataframes, they refer to the same matches, but just have different stats
team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")

In [22]:
''' 
Scraping Data For Multiple Seasons and Teams with a Loop (scaling up)
'''

' \nScraping Data For Multiple Seasons and Teams with a Loop (scaling up)\n'

In [23]:
# We have taken the matches dataframe, and added some of the shooting dataframe columns to the end
team_data.head()

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,...,Opp Formation,Referee,Match Report,Notes,Sh,SoT,Dist,FK,PK,PKatt
0,2024-08-18,16:30,Premier League,Matchweek 1,Sun,Home,L,0.0,2.0,Manchester City,...,3-2-4-1,Anthony Taylor,Match Report,NaN,10,3,17.1,0.0,0,0
1,2024-08-22,20:00,Conf Lg,Play-off round,Thu,Home,W,2.0,0.0,ch Servette FC,...,4-2-3-1,Jérémie Pignard,Match Report,Leg 1 of 2,14,7,NaN,NaN,1,1
2,2024-08-25,14:00,Premier League,Matchweek 2,Sun,Away,W,6.0,2.0,Wolves,...,4-2-3-1,Darren England,Match Report,NaN,14,8,15.4,1.0,0,0


In [24]:
# Setting up a list of the years we want to scrape [2025, 2024]
years = list(range(2025, 2023,-1))
# years = list(range(2024, 2022), -1)
# The list will contain several dataframes, each dataframe will contain the match logs for 1 team in 1 season
all_matches = []

In [25]:
# define the url we want to start on
standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"

In [26]:
import time

# loop through the list of years 
for year in years:

    # temporary comment
    print("Year: ", year, "\n")
    
    # retrieving the home page html for a season's standings 
    data = requests.get(standings_url)

    # do nothing for 10 seconds after retrieving the html for the season standings
    #time.sleep(10)
    time.sleep(3)
    
    soup = BeautifulSoup(data.text)
    # Obtain the stats table, which contains all of our individual team links, which give us the individual match data for each team
    standings_table = soup.select('table.stats_table')[0]

    # find all of the team links, and grab the href property
    links = [l.get("href") for l in standings_table.find_all('a')]
    # filter the links so we only have the links for the squads
    links = [l for l in links if '/squads/' in l]
    # then we turn the links from relative to absolute links
    team_urls = [f"https://fbref.com{l}" for l in links]

    # grab the url for the previous season, for the next loop iteration guarentees we will get the standings_url
    # for the next season and allow us to scrape data for the next season
    previous_season = soup.select("a.prev")[0].get("href")
    standings_url = f"https://fbref.com{previous_season}"

    # loop through each of the team urls, and individually scrape the match logs for each team
    # Example team url: 'https://fbref.com/en/squads/b8fd03ef/2023-2024/Manchester-City-Stats'
    for team_url in team_urls:
        # getting the team name by grabbing the last portion of the team_url, also getting rid of "-Stats" and "-"
        # Example result: Manchester City (setting the team name properly)
        team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")
        # get the team_url, and read that into matches
        data = requests.get(team_url)

        # do nothing for 10 seconds after retrieving the html for the specific team's page
        #time.sleep(10)
        time.sleep(5)
        #time.sleep(4)
        
        # matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
        html_content = StringIO(data.text)

        # temporary output code
        print("Team Scores & Fixtures: ", team_name, "\n")
        
        matches = pd.read_html(html_content, match="Scores & Fixtures")[0]
        
        # retrieving the all_comps/shooting link, only finding the link to the shooting stats (are duplicate links)
        soup = BeautifulSoup(data.text)
        links = [l.get("href") for l in soup.find_all('a')]
        links = [l for l in links if l and 'all_comps/shooting/' in l]
        # converting the link to an absolute url, grabbing the html for the specific shooting stats page
        data = requests.get(f"https://fbref.com{links[0]}")

        # do nothing for 10 seconds after retrieving the html for the specific team's shooting page
        #time.sleep(10)
        time.sleep(5)
        #time.sleep(4)

        # temporary output code
        print("Team Shooting Stats: ", team_name, "\n")
        
        # reading in our shooting stats to a pandas dataframe
        html_content = StringIO(data.text)
        shooting = pd.read_html(html_content, match="Shooting")[0]
        # drop the top level index
        shooting.columns = shooting.columns.droplevel()

        # attempt to merge, sometimes for some teams the shooting stats are not available, and when you
        # try to merge the 2, pandas can give you a value error (because shooting stats dataframe is empty)
        # if this occurs, ignore team and continue with the loop (skipping over teams where stats aren't available)
        try:
            # Combine the "matches" and "shooting" dataframes, they refer to the same matches, but just have different stats
            team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")
        except ValueError:
            continue

        # filter only for premier league games
        team_data = team_data[team_data["Comp"] == "Premier League"]
        # adding "Season" and "Team" column to help us diffrentiate between seasons and teams
        team_data["Season"] = year
        team_data["Team"] = team_name
        all_matches.append(team_data)
        # does nothing for 10 seconds, doing this because a lot of websites do not want people
        # to scrape to quickly because it may slow down their website, by slowing down how we
        # are scraping we are making sure we don't get blocked from scraping the website (so we
        # don't make a lot of requests very quickly)

        # just do nothing for 10 additional seconds just in case
        # time.sleep(10)


Year:  2025 

Team Scores & Fixtures:  Manchester City 

Team Shooting Stats:  Manchester City 

Team Scores & Fixtures:  Brighton and Hove Albion 

Team Shooting Stats:  Brighton and Hove Albion 

Team Scores & Fixtures:  Arsenal 

Team Shooting Stats:  Arsenal 

Team Scores & Fixtures:  Liverpool 

Team Shooting Stats:  Liverpool 

Team Scores & Fixtures:  Tottenham Hotspur 

Team Shooting Stats:  Tottenham Hotspur 

Team Scores & Fixtures:  Newcastle United 

Team Shooting Stats:  Newcastle United 

Team Scores & Fixtures:  Nottingham Forest 

Team Shooting Stats:  Nottingham Forest 

Team Scores & Fixtures:  Chelsea 

Team Shooting Stats:  Chelsea 

Team Scores & Fixtures:  West Ham United 

Team Shooting Stats:  West Ham United 

Team Scores & Fixtures:  Manchester United 

Team Shooting Stats:  Manchester United 

Team Scores & Fixtures:  Fulham 

Team Shooting Stats:  Fulham 

Team Scores & Fixtures:  Aston Villa 

Team Shooting Stats:  Aston Villa 

Team Scores & Fixtures:  Bre

In [27]:
len(all_matches)

40

In [29]:
# combine all of our individual dataframes into one dataframe, takes in a list of dataframes and
# return a single dataframe
match_df = pd.concat(all_matches)

In [30]:
match_df

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,...,Match Report,Notes,Sh,SoT,Dist,FK,PK,PKatt,Season,Team
1,2024-08-18,16:30,Premier League,Matchweek 1,Sun,Away,W,2,0,Chelsea,...,Match Report,NaN,11.0,5.0,19.1,0.0,0,0,2025,Manchester City
2,2024-08-24,15:00,Premier League,Matchweek 2,Sat,Home,W,4,1,Ipswich Town,...,Match Report,NaN,13.0,4.0,17.8,1.0,1,1,2025,Manchester City
0,2024-08-17,15:00,Premier League,Matchweek 1,Sat,Away,W,3.0,0.0,Everton,...,Match Report,NaN,10.0,5.0,13.8,0.0,0,0,2025,Brighton and Hove Albion
1,2024-08-24,12:30,Premier League,Matchweek 2,Sat,Home,W,2.0,1.0,Manchester Utd,...,Match Report,NaN,14.0,4.0,14.2,1.0,0,0,2025,Brighton and Hove Albion
0,2024-08-17,15:00,Premier League,Matchweek 1,Sat,Home,W,2.0,0.0,Wolves,...,Match Report,NaN,18.0,6.0,17.3,0.0,0,0,2025,Arsenal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36,2024-04-24,20:00,Premier League,Matchweek 29,Wed,Away,L,2,4,Manchester Utd,...,Match Report,NaN,10.0,4.0,17.8,1.0,0,0,2024,Sheffield United
37,2024-04-27,15:00,Premier League,Matchweek 35,Sat,Away,L,1,5,Newcastle Utd,...,Match Report,NaN,15.0,4.0,13.5,0.0,0,0,2024,Sheffield United
38,2024-05-04,15:00,Premier League,Matchweek 36,Sat,Home,L,1,3,Nott'ham Forest,...,Match Report,NaN,16.0,4.0,18.0,0.0,1,1,2024,Sheffield United
39,2024-05-11,15:00,Premier League,Matchweek 37,Sat,Away,L,0,1,Everton,...,Match Report,NaN,13.0,1.0,21.0,0.0,0,0,2024,Sheffield United


In [31]:
match_df.columns

Index(['Date', 'Time', 'Comp', 'Round', 'Day', 'Venue', 'Result', 'GF', 'GA',
       'Opponent', 'xG', 'xGA', 'Poss', 'Attendance', 'Captain', 'Formation',
       'Opp Formation', 'Referee', 'Match Report', 'Notes', 'Sh', 'SoT',
       'Dist', 'FK', 'PK', 'PKatt', 'Season', 'Team'],
      dtype='object')

In [30]:
# optional: if you prefer to have all columns be lowercase, replaces all column names
match_df.columns = [c.lower() for c in match_df.columns]

In [31]:
# write this data to a csv file
match_df.to_csv("matches.csv")